In [2]:
import cv2
import os
import pydicom



In [ ]:
### convert dicom to png
# train set
rdir = '../ProjectPneumonia/rsna-pneumonia-detection-challenge/stage_2_train_images/'
wdir = '../ProjectPneumonia/rsna-pneumonia-detection-challenge/train/'

train_list = [ img for img in os.listdir(rdir)]
for img in train_list:  
    ds = pydicom.read_file(rdir + img) 
    image = ds.pixel_array 
    cv2.imwrite(wdir + img.replace('.dcm','.png'),image)
    
# test set
rdir = '../ProjectPneumonia/rsna-pneumonia-detection-challenge/stage_2_test_images/'
wdir = '../ProjectPneumonia/rsna-pneumonia-detection-challenge/test/'

test_list = [ img for img in os.listdir(rdir)]
for img in test_list[20:]:  
    ds = pydicom.read_file(rdir + img) 
    image = ds.pixel_array 
    cv2.imwrite(wdir + img.replace('.dcm','.png'),image)


In [3]:
import pandas as pd

In [4]:
train_labels = pd.read_csv('../ProjectPneumonia/rsna-pneumonia-detection-challenge/stage_2_train_labels.csv')
train_labels.head()



,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1


In [6]:
label_image = {}
def lab_img(df):
    for n, row in df.iterrows():
            pid = row['patientId']
            if pid not in label_image:
                label_image[pid] = {
                    'image': '../ProjectPneumonia/rsna-pneumonia-detection-challenge/train/%s.png' % pid,
                    'label': row['Target'],
                    }
    return label_image
        

In [7]:
trainset = lab_img(train_labels) 
trainset = pd.DataFrame(trainset)

In [8]:
x_train = trainset.iloc[0]
y_train = trainset.iloc[1]

In [11]:
X_train = []
for imagepath in x_train: 
    image = cv2.imread(imagepath)
    image = cv2.resize(image, (28, 28))
    X_train.append(image)

In [15]:
import numpy as np
X_train = np.array(X_train, dtype="float") / 255.0

In [18]:
y_train = np.array(y_train)

In [21]:
X_train.shape

(26684, 28, 28, 3)

In [22]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)


Using TensorFlow backend.


In [33]:
from sklearn.model_selection import train_test_split
(trainX, testX, trainY, testY) = train_test_split(X_train,y_train, test_size=0.2, random_state=42)

In [30]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

#create model
model = Sequential()
#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu',input_shape=(28,28,3)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))

In [31]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [35]:
model.fit(trainX, trainY,validation_data=(testX, testY),epochs=3)

Train on 21347 samples, validate on 5337 samples
Epoch 1/3
21347/21347 [==============================] - 38s 2ms/step - loss: 0.4079 - accuracy: 0.8146 - val_loss: 0.4042 - val_accuracy: 0.8175
Epoch 2/3
21347/21347 [==============================] - 39s 2ms/step - loss: 0.4015 - accuracy: 0.8200 - val_loss: 0.4100 - val_accuracy: 0.8141
Epoch 3/3
21347/21347 [==============================] - 38s 2ms/step - loss: 0.3950 - accuracy: 0.8216 - val_loss: 0.4051 - val_accuracy: 0.8154


In [39]:
score = model.evaluate(testX, testY, verbose=1)

5337/5337 [==============================] - 2s 350us/step


In [41]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.4051007312045956
Test accuracy: 0.8154394030570984
